Парсинг названия криптовалюты, ее цены, просадки или роста за последние 24 часа и 7 дней (в процентах) и капитализации рынка валюты. А также положим данные в pandas датафрейм, предварительно сохранив в csv файл.

In [1]:
import requests
from bs4 import BeautifulSoup
import csv


def get_html(url):
    r = requests.get(url)
    return r.text


def get_total_pages(html):
    soup = BeautifulSoup(html, 'lxml')
    
    pages = soup.find('ul', class_='pagination').find_all('a', role='button')[-2].get('href')
    total_pages = pages.split('=')[1]
    
    return total_pages


def write_csv(data):
    with open('coinmarketcup_many_pages.csv', 'a') as f:
        writer = csv.writer(f)
        
        writer.writerow((data['Name'],
                         data['Price'],
                         data['24h %'],
                         data['7d %'],
                         data['Market Cap']))


def get_page_data(html):
    soup = BeautifulSoup(html, 'lxml')
    
    cript = soup.table.tbody.find_all('tr')
    
    for cr in cript:
        # name, price, 24h, 7d, market_cap
        try:
            try:
                name = cr.find('div', class_='sc-16r8icm-0 escjiH').find('p').text
            except:
                name = cr.find_all('span')[3].text
        except:
            name = ''
        
        try:
            try:
                price = cr.find('div', class_='sc-131di3y-0 cLgOOr').text
            except:
                price = cr.find_all('span')[5].text
        except:
            price = ''
            
        try:
            procent = cr.find_all('span', class_='sc-15yy2pl-0 hzgCfk')
            if procent[0].find('span', class_="icon-Caret-up") == None:
                _24h = '-' + procent[0].text
            else:
                _24h = procent[0].text
        except:
            _24h = ''
            
        try:
            procent = cr.find_all('span', class_='sc-15yy2pl-0 hzgCfk')
            if procent[1].find('span', class_="icon-Caret-up") == None:
                _7d = '-' + procent[1].text
            else:
                _7d = procent[1].text
        except:
            _7d = ''
            
        try:
            market_cap = cr.find('span', class_='sc-1ow4cwt-0 iosgXe').text
        except:
            market_cap = ''
            
        data = {'Name': name,
                'Price': price,
                '24h %': _24h,
                '7d %': _7d,
                'Market Cap': market_cap}
        
        write_csv(data)

        
def main():
    url = 'https://coinmarketcap.com/'
    base_url = 'https://coinmarketcap.com/?'
    page_part = 'page='
    total_page = get_total_pages(get_html(url))
    
#   Для всех страниц. 
#   for p in range(1, int(total_pages)+1):         

#   Чтобы долго не ожидать результат парсинга всех страниц, возьмем 5 (для примера)
    for p in range(1, 6):
        url_gen = base_url + page_part + str(p)
        html = get_html(url_gen)
        
        get_page_data(html)


if __name__ == '__main__':
    main()

In [2]:
import pandas as pd

data = pd.read_csv('coinmarketcup_many_pages.csv', names=['Name', 'Price', '24h %', '7d %', 'Market Cap'])
data

,Name,Price,24h %,7d %,Market Cap
0,Bitcoin,"$31,416.80",-5.25%,-18.25%,$598.05B
1,Ethereum,"$2,380.05",-1.31%,-15.89%,$287.32B
2,Tether,$0.9999,-0.01%,NaN,$83.16B
3,BNB,$321.33,-4.51%,-17.05%,$52.47B
4,USD Coin,$1.00,NaN,NaN,$48.44B
...,...,...,...,...,...
495,Harvest Finance,$63.68,NaN,NaN,NaN
496,Kava Swap,$1.26,NaN,NaN,NaN
497,RIZON,$0.02,NaN,NaN,NaN
498,Velo,$0.03,NaN,NaN,NaN


In [3]:
data.isna().sum()

Name            0
Price           0
24h %         453
7d %          460
Market Cap    450
dtype: int64